In [219]:
import math
import sys
import pprint
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt

pp = pprint.PrettyPrinter()

In [220]:
def readfile(filename):
    with open(filename) as f:
        lines = f.readlines()
    return lines

In [248]:
def process_original_data(original):
    original = [data for data in original if data.find('m3') != -1]
    
    modified = {}
    for org in original:
        moteids = org[org.find(',') + 1:org.find(';')]
        data = org[org.find(';') + 1:]
#         print org, moteids, data
        for mote in moteids.split('+'):
            modified['m3-' + str(mote)] = data.strip()
    
    return modified

def add_sink_data(sinklog, datamap):
    sinklog = filter(lambda x: x.startswith('Sink got message from'), sinklog)
    for log in sinklog:
        message = log.split(' ')[8][1:-1]
            
        if (message not in datamap.keys()):
            datamap[message] = {
                'start_time': [],
                'finish_time': [],
                'min_start_time': 25000,
                'max_finish_time': 0,
            }
        
        ftime = int(log.split(' ')[10][5:])
        if ftime not in datamap[message]['finish_time']:
            datamap[message]['finish_time'].append(ftime)
        datamap[message]['max_finish_time'] = max(datamap[message]['finish_time'])
    return datamap

def add_send_data(sendlog, datamap, original):
    sendlog = filter(lambda x: x.find('Sending on time=') != -1, sendlog)
    for log in sendlog:
        log = log.strip()
        moteid = log.split(';')[1]
        message = original[moteid]
        stime = int(log.split(';')[2].split(' ')[2][5:])
        if stime not in datamap[message]['start_time']:
            datamap[message]['start_time'].append(stime)
        datamap[message]['min_start_time'] = min(datamap[message]['start_time'])
    return datamap

# def update_max_min(datamap):
#     for k in datamap.keys():
#         datamap[k]['min_start_time'] = min(datamap[k]['start_time'])
#         datamap[k]['max_finish_time'] = max(datamap[k]['finish_time'])
#     return datamap

def report_agg(sendlog):
    sendlog = filter(lambda x: x.find('Number of aggregated packets are=') != -1, sendlog)
    count_agg = {}
    for log in sendlog:
        moteid = log.split(';')[1]
        count_agg[moteid] = 0
    for log in sendlog:
        moteid = log.split(';')[1]
        acount = int(log.split(';')[2][33:])
        count_agg[moteid] = max(acount, count_agg[moteid])
    count_agg = {k:v for k,v in count_agg.items() if v != 0}
    print('\nAggregation Map: ')
    pp.pprint(count_agg)
    print('Total aggregation = ' + str(sum(count_agg.values())) + '\n\n')
    
def parse_data_collect(folders):
    for folder in folders:
        datamap = {}
        sinklog = readfile(folder + 'Sink.txt')
        sendlog = readfile(folder + 'Sources.txt')
        original = readfile(folder + 'Data.txt')
        
        original = process_original_data(original)
        datamap = add_sink_data(sinklog, datamap)
        datamap = add_send_data(sendlog, datamap, original)
        pp.pprint(datamap)
        
def sink_stats(sinklog):
    sinklog = filter(lambda x: x.startswith('Sink got message from'), sinklog)
    messages = [log.split(' ')[8][1:-1] for log in sinklog]
    sink_count = {message:messages.count(message) for message in messages}
    print('\nSink Stats: ')
    pp.pprint(sink_count)
    print('Total sink got = ' + str(sum(sink_count.values())) + '\n\n')
        
def print_stats(sendlog, sinklog):
    report_agg(sendlog)
    sink_stats(sinklog)
        
def parse_data_algo(folders):
    for folder in folders:
        
        datamap = {}
        sinklog = readfile(folder + 'Sink.txt')
        sendlog = readfile(folder + 'Sources.txt')
        original = readfile(folder + 'Data.txt')
        
        original = process_original_data(original)
        datamap = add_sink_data(sinklog, datamap)
        datamap = add_send_data(sendlog, datamap, original)
        pp.pprint(datamap)
        
        print_stats(sendlog, sinklog)

In [249]:
if __name__ == "__main__":
#     folders = ['cci8motes50collect/']
#     parse_data_collect(folders)
    folders = ['cci8motes50dfs/']
    parse_data_algo(folders)

{'0hello': {'finish_time': [90, 95, 243, 245],
            'max_finish_time': 245,
            'min_start_time': 82,
            'start_time': [82, 233]},
 '1hello': {'finish_time': [99, 101, 253],
            'max_finish_time': 253,
            'min_start_time': 93,
            'start_time': [93, 245, 246]},
 '2hello': {'finish_time': [107, 111, 257, 263],
            'max_finish_time': 263,
            'min_start_time': 106,
            'start_time': [107, 106, 257, 258]},
 '3hello': {'finish_time': [125, 277],
            'max_finish_time': 277,
            'min_start_time': 117,
            'start_time': [117, 268]},
 '4hello': {'finish_time': [132, 284],
            'max_finish_time': 284,
            'min_start_time': 128,
            'start_time': [128, 279]},
 '5hello': {'finish_time': [146, 297],
            'max_finish_time': 297,
            'min_start_time': 141,
            'start_time': [141, 292]},
 '6hello': {'finish_time': [165, 314],
            'max_finish_time': 314